In [1]:
#essentials
import numpy as np
import pandas as pd

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import shap
import ast

#tools/metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

#modeling
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge


In [2]:
kp =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KenPom Team Metrics.csv")
kp_conf_rat =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\KP_Conf_Ratings.csv")

df =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\training_data_to_use.csv")
models_parameters =  pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\model_parameters_defense.csv")

model_hyperparameters = pd.read_csv("D:\\Github\\Capstone-project\\Data folder\\model_hyper_parameters_defense.csv")

In [3]:
df.head(3)

,Unnamed: 0,player_type,Team,Team_n1,Unnamed..0_x,Player.Name,season,conf_abbr,games,games_started,...,Calc B,Calc B_n1,Calc C,Calc C_n1,Calc Overall A,Calc Overall A2,Calc Overall A3,Calc Overall A_n1,Calc Overall A2_n1,Calc Overall A3_n1
0,1,1,JACKSONVILLE,JACKSONVILLE,16199.0,aamahne-santos-1,2018-19,A-Sun,32,31,...,513.80,618.80,-800.06,-516.80,594.54,168.82,7450.10,720.80,360.40,6834.0
1,2,1,CLEMSON,CLEMSON,7084.0,aamir-simms-1,2018-19,ACC,34,34,...,1100.55,1676.10,493.58,1155.15,1280.64,1374.02,8037.35,1985.65,2272.55,9769.7
2,3,1,CLEMSON,CLEMSON,7085.0,aamir-simms-1,2019-20,ACC,30,30,...,1676.10,1453.86,1155.15,1461.24,1985.65,2272.55,9769.70,1719.54,2095.92,9778.5


In [4]:
year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'), 
             (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
year_val = [1, 2, 3, 4]
df['Class_n1'] = np.select(year_cond, year_val)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*(df['ws'])
df['Calc A2'] = df['X.Min']*df['bpm']
# df['Calc A2'] = df['bpm']/(1-df['X.Min'])
df['Calc A3'] = df['X.Min']*df['ORtg']
df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
# df['Calc A2_n1'] = df['bpm_n1']/(1-df['X.Min_n1'])
df['Calc A_n1'] =df['X.Min_n1']*(df['ws_n1'])
df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']

In [49]:
models_parameters.head(3)

,regression_model,parameter_list,no_of_features
0,linear_regression,"['player_type','games', 'games_started', 'mp_p...",4
1,ridge_regression,"['player_type', 'games', 'fg_per_g', 'fga_per_...",4
2,xg_boost,"['player_type', 'trb_per_min', 'trb_per_poss',...",4


In [50]:
columns1 = ['player_type','games','games_started','mp_per_g','sos','trb_per_poss','stl_per_poss','blk_per_poss',
            'pf_per_poss','def_rtg','per','orb_pct','drb_pct','trb_pct','stl_pct',
            'Class_n1', 'Year_dif', 'Team_AdjEM_dif', 'KP_Conf_Dif', 'KP_Rk_Dif',
            'blk_pct','dws','ws','ws_per_40','dbpm','bpm','Ht','Wt','DR.','Blk.','Stl.','FC.40']

In [51]:
model_hyperparameters.head(3)

,model,best_score,best_params
0,linear_regression,0.630944,{}
1,ridge_regression,0.634638,{'alpha': 150}
2,xg_boost,0.616541,"{'learning_rate': 0.1, 'max_depth': 4, 'min_ch..."


In [52]:
def r2(actual: np.ndarray, predicted: np.ndarray):
    """ R2 Score """
    return r2_score(actual, predicted)

def adjr2(actual: np.ndarray, predicted: np.ndarray, rowcount: int, featurecount: int):
    """ Adjusted R2 Score """
    return 1-(1-r2(actual,predicted))*(rowcount-1)/(rowcount-featurecount)

In [53]:
columns = models_parameters['parameter_list'][0]
columns = ast.literal_eval(columns)
print(len(columns))

115


In [55]:
model_hyperparameters['model'][2]

'xg_boost'

In [56]:
hyperparameters = model_hyperparameters['best_params'][2]
hyperparameters = ast.literal_eval(hyperparameters)
print(type(hyperparameters))
print(hyperparameters)

<class 'dict'>
{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 0.1, 'subsample': 0.5}


In [39]:
master_table  = df[columns]
print(master_table.shape)
    
y= (df['defence_def_rtg']).to_frame()
print(y.shape)

(7747, 115)
(7747, 1)


In [40]:
# split the data with 90% in training set
X_train, X_test, y_train, y_test = train_test_split(master_table, y, random_state=0,
                                    train_size=0.85)

In [60]:

model_xg = XGBRegressor(learning_rate= 0.1, max_depth= 4, min_child_weight = 0.1, subsample= 0.5)
model_xg.fit(X_train, y_train)
y_predicted = model_xg.predict(X_test)
print(y_predicted.shape)


(1163,)


In [42]:

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_predicted = model_lr.predict(X_test)
print(y_predicted.shape)


(1163, 1)


In [61]:
y_predicted.shape[0]

1163

In [62]:
no_of_features = len(columns)
no_of_rows = y_predicted.shape[0]

print(adjr2(y_test,y_predicted,no_of_rows, no_of_features ))
print(r2(y_test,y_predicted))

#0.6207673008133114


0.6158333749671044
0.6535226996260977
